<a href="https://colab.research.google.com/github/termls/TextSummarization/blob/main/TextSummarization4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import nltk
import re

In [ ]:
nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
def preprocess_text(text):
    import unicodedata
    text = unicodedata.normalize("NFKC", text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    sentences = nltk.sent_tokenize(text)
    text = " ".join(sentences)
    return text

In [ ]:
#def preprocess_pair(examples):
    #examples['document'] = [preprocess_text(doc) for doc in examples['document']]
    #examples['summary'] = [preprocess_text(summ) for summ in examples['summary']]
    #return examples

def preprocess_pair(examples):
    return {
        "input": [preprocess_text(doc) for doc in examples["article"]],
        "target": [preprocess_text(summ) for summ in examples["highlights"]],
    }


In [ ]:
!pip uninstall -y datasets
!pip install datasets --upgrade


Found existing installation: datasets 2.14.4
Uninstalling datasets-2.14.4:
  Successfully uninstalled datasets-2.14.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine

In [ ]:
from datasets import load_dataset

# Load the full dataset
dataset = load_dataset("cnn_dailymail", '3.0.0')

train_set = dataset["train"].shuffle(seed=42)
val_set = dataset["validation"].shuffle(seed=42)

# Apply preprocessing
train_set = train_set.map(preprocess_pair, batched=True)
val_set = val_set.map(preprocess_pair, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding="max_length")

    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels_example]
        for labels_example in labels["input_ids"]
    ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_tokenized = train_set.map(tokenize_function, batched=True, remove_columns=train_set.column_names, num_proc=4)
val_tokenized = val_set.map(tokenize_function, batched=True, remove_columns=val_set.column_names)

Map (num_proc=4):   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

In [ ]:
!pip install --upgrade transformers

In [ ]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5599e216e79c6dca0f4f940ea93c728422a4f821c59d6f44919b865bf190cf79
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Now values are floats directly, multiply by 100 for percentages
    result = {key: value * 100 for key, value in result.items()}

    result["gen_len"] = np.mean([len(pred.split()) for pred in decoded_preds])

    return result

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./bart-large-cnn",
    num_train_epochs=3,
    warmup_steps=500,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=2,
    learning_rate=3e-5,
    max_grad_norm=1.0,
    weight_decay=0.01,
    save_strategy="epoch",
    eval_strategy="epoch",
    save_total_limit=2,
    predict_with_generate=True,
    fp16=True,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    report_to="none",
)

In [ ]:
model = BartForConditionalGeneration.from_pretrained(
    "/content/drive/MyDrive/bart-large-cnn/checkpoint-71780"
)

/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [ ]:
from transformers import EarlyStoppingCallback

def model_init():
    return BartForConditionalGeneration.from_pretrained(
        "/content/drive/MyDrive/bart-large-cnn/checkpoint-71780"
    )

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train(resume_from_checkpoint="/content/drive/MyDrive/bart-large-cnn/checkpoint-71780")

<ipython-input-16-1fad2bc3f986>:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
2,1.077500,1.591169,45.440919,21.855981,31.176509,42.535727,70.127693


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=107667, training_loss=0.3633532314847407, metrics={'train_runtime': 37688.638, 'train_samples_per_second': 22.854, 'train_steps_per_second': 2.857, 'total_flos': 9.33287435050156e+17, 'train_loss': 0.3633532314847407, 'epoch': 2.9999930341258176})

In [ ]:
output_dir = "/content/drive/MyDrive/bart-large-finetuned2"

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/bart-large-finetuned2/tokenizer_config.json',
 '/content/drive/MyDrive/bart-large-finetuned2/special_tokens_map.json',
 '/content/drive/MyDrive/bart-large-finetuned2/vocab.json',
 '/content/drive/MyDrive/bart-large-finetuned2/merges.txt',
 '/content/drive/MyDrive/bart-large-finetuned2/added_tokens.json')

In [ ]:
trainer.save_model("./bart-base-finetuned")
tokenizer.save_pretrained("./bart-base-finetuned")

('./bart-base-finetuned/tokenizer_config.json',
 './bart-base-finetuned/special_tokens_map.json',
 './bart-base-finetuned/vocab.json',
 './bart-base-finetuned/merges.txt',
 './bart-base-finetuned/added_tokens.json')

In [ ]:
device = "cpu"
model = model.to(device)

def summarize(text, max_input_length=1024, max_target_length=128):
    inputs = tokenizer(
        text,
        max_length=max_input_length,
        return_tensors="pt",
        truncation=True,
        padding="max_length"
    ).to(model.device)

    summary_ids = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=max_target_length,
    num_beams=4,
    early_stopping=True,
    do_sample=False,
    length_penalty=1.0  # try 1.2 if output is too short

    )


    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

model_path = "/content/drive/MyDrive/bart-large-finetuned/"
tokenizer = BartTokenizer.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path)

/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [ ]:
text = input()

print("\nSummary:\n", summarize(text))

Team radio revealed that Stroll, who was not on a fast lap, was told Leclerc was approaching but the Canadian said: “I didn’t hear you.”  Stroll sustained damage to his rear suspension and his session was over as Aston Martin changed his gearbox. He was later given a one-place grid penalty by the stewards.  Watching on from the Aston Martin garage was Adrian Newey, who is at the track for the first time since starting work as managing technical partner in March.  The 66-year-old, widely considered as the greatest technical mastermind in the sport, was pictured inspecting the car with his trademark notebook.  Leclerc doubted his Ferrari would excel in the slow-speed corners of Monaco but showed strong pace after receiving a new front wing to finish 0.163 seconds ahead of Max Verstappen.

Summary:
 Stroll sustained damage to his rear suspension and his session was over .
He was given a one-place grid penalty by the stewards .
Adrian Newey is at the track for the first time since starting